In [15]:
import os.path
import cv2
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import src.metrics as nc_metrics
import src.utils as nc_utils
import notebooks.sandbox.models as nc_models

import time

from pathlib import Path
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import models

In [16]:
strategy = nc_utils.start_session()

ROOT_DIR = "src/data"
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
MASK_DIR = os.path.join(ROOT_DIR, "masks")

BATCH_SIZE = 4
LR = 1e-4  # Learning rate
LR_FINE = 1e-6 # Fine-tuning learning rate
EPOCHS = 300

smallest_dimension = nc_utils.get_smallest_image_dimension(IMAGE_DIR)

#IMAGE_SIZE = smallest_dimension
IMAGE_SIZE = 256
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)
INPUT_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

In [17]:
(x_train_paths, y_train_paths), (x_valid_paths, y_valid_paths), (
x_test_paths,
y_test_paths) = nc_utils.load_data(
    image_directory=IMAGE_DIR,
    mask_directory=MASK_DIR,
    split=0.1
)

train_dataset = nc_utils.get_tensorflow_dataset(
    image_mask_paths=(x_train_paths, y_train_paths),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

validation_dataset = nc_utils.get_tensorflow_dataset(
    image_mask_paths=(x_valid_paths, y_valid_paths),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

test_dataset = nc_utils.get_tensorflow_dataset(
    image_mask_paths=(x_test_paths, y_test_paths),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

data_aug = nc_utils.get_data_augmentation_pipeline()

In [18]:
# Initiating model on GPU
with strategy.scope():
    model = nc_models.mobilenetv2_unet_model(input_shape=INPUT_SHAPE, augment_data=True)
    metrics=[nc_metrics.continuous_dice_coefficient, nc_metrics.dice_coefficient, metrics.MeanIoU(2), metrics.RootMeanSquaredError(),
             nc_metrics.bahd, nc_metrics.surface_distance_metric, metrics.Recall(), metrics.Precision(), metrics.Accuracy()]
    loss=nc_metrics.cdc_loss
    opt=tf.keras.optimizers.Nadam(LR)

2024-02-17 16:07:15.294667: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 13.5KiB (rounded to 13824)requested by op AddV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-02-17 16:07:15.294701: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-02-17 16:07:15.294716: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 101, Chunks in use: 101. 25.2KiB allocated for chunks. 25.2KiB in use in bin. 7.4KiB client-requested in use in bin.
2024-02-17 16:07:15.294728: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 49, Chunks in use: 49. 30.5KiB allocated for chunks. 30.5KiB in use in bin. 23.0KiB client-requested in use in bin.
2024-02-17 16:07:15.294739:

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2] name: 

In [ ]:
model.summary()

In [ ]:
# Compiling model
model.compile(optimizer=opt,
              loss=loss,
              metrics=metrics)

early_stopping = callbacks.EarlyStopping(monitor='val_loss',
                                         patience=10,
                                         restore_best_weights=True)
reduce_learning = callbacks.ReduceLROnPlateau(monitor='val_loss',
                                              factor=0.1,
                                              patience=4)

In [ ]:
train_steps = len(x_train_paths) // BATCH_SIZE
valid_steps = len(x_valid_paths) // BATCH_SIZE

if len(x_train_paths) % BATCH_SIZE != 0:
    train_steps += 1
if len(x_valid_paths) % BATCH_SIZE != 0:
    valid_steps += 1



try:
    initial_time = time.time()
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=validation_dataset,
        callbacks=[early_stopping, reduce_learning],
        steps_per_epoch=train_steps,
        validation_steps=valid_steps)
    final_time = time.time()
    total_train_time = final_time - initial_time
except Exception as e:
    print("An error occurred:", e)

In [ ]:
print(total_train_time)

In [ ]:
# Fine-Tune
fine_tune_at = int(nc_models.MOBILENETV2_SIZE * 0.7)
for layer in model.layers[fine_tune_at:nc_models.MOBILENETV2_SIZE]:
    layer.trainable = True

In [ ]:
# Compiling model
opt = tf.keras.optimizers.Nadam(LR_FINE)
model.compile(optimizer=opt,
              loss=loss,
              metrics=metrics)

early_stopping = callbacks.EarlyStopping(monitor='val_loss',
                                         patience=10,
                                         restore_best_weights=True)
reduce_learning = callbacks.ReduceLROnPlateau(monitor='val_loss',
                                              factor=0.1,
                                              patience=4)

In [ ]:
try:
    initial_time = time.time()
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=validation_dataset,
        callbacks=[early_stopping, reduce_learning],
        steps_per_epoch=train_steps,
        validation_steps=valid_steps)
    final_time = time.time()
    total_train_time = final_time - initial_time
except Exception as e:
    print("An error occurred:", e)

In [ ]:
print(total_train_time)

In [ ]:
test_steps = len(x_test_paths) // BATCH_SIZE
if len(x_test_paths) % BATCH_SIZE != 0:
    test_steps += 1

initial_time = time.time()
model.evaluate(test_dataset, steps=test_steps)
final_time = time.time()

total_eval_time = final_time - initial_time

In [ ]:
print(total_eval_time)

# Results

In [ ]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = x/255.0
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = np.expand_dims(x, axis=-1)
    x = x/255.0
    return x

In [ ]:
def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

In [ ]:
for i, (x, y) in enumerate(zip(x_test_paths[:10], y_test_paths[:10])):
    x = read_image(x)
    y = read_mask(y)
    y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
    h, w, _ = x.shape
    white_line = np.ones((h, 10, 3))

    all_images = [
        x, white_line,
        mask_parse(y), white_line,
        mask_parse(y_pred)
    ]
    image = np.concatenate(all_images, axis=1)

    fig = plt.figure(figsize=(12, 12))
    a = fig.add_subplot(1, 1, 1)
    imgplot = plt.imshow(image)